# Google Colab version

## 1. set up environment on Colab

In [ ]:
!pip install bitsandbytes

### 1.1 Download full dataset

In [ ]:
# Define the URL of the file you want to download
# file_url = "https://codeload.github.com/SaFoLab-WISC/JailBreakV_28K/zip/refs/heads/V0.2"
file_url = "https://drive.usercontent.google.com/download?id=1ZrvSHklXiGYhpiVoxUH8FWc5k0fv2xVZ&export=download&authuser=0&confirm=t&uuid=0e44de7a-77d4-4b81-a953-1208d111221c&at=AN8xHooGPXhnTlQjNRWrPFLvCaSJ%3A1758342827507"

# Define the destination path in Google Drive
# Make sure the folder 'Colab Notebooks/' exists in your Google Drive
destination_path = ""

# Use !wget to download the file directly to the specified path
!wget "{file_url}" -P "{destination_path}" --no-check-certificate

### 1.2 unzip full dataset

In [ ]:
# --- Define your paths ---

# 1. output_path: Where the zip file is saved after download
#    (The exact path depends on where you want it in 'My Drive')
output_path = "/content/drive/MyDrive/Colab Notebooks/JailBreakV_28K.zip"

# 2. extract_dir: The destination folder for the unzipped contents
#    (Again, define where in 'My Drive' you want the files to go)
extract_dir = "/content/drive/MyDrive/Colab Notebooks/JailBreakV_28K"

# --- Execute the unzip command ---

# The command uses the values stored in the Python variables above
!unzip -q "{output_path}" -d "{extract_dir}"


## 2. Import libraries

In [ ]:
import os
import io
import time
import json
import torch
import pandas as pd
import re # Added regex import
from PIL import Image
from pydantic import BaseModel, Field, ValidationError # Added ValidationError import
from transformers import AutoProcessor, AutoModelForVision2Seq

In [ ]:
# Check data
base_data_path = "/content/drive/My Drive/Colab Notebooks/JailBreakV_28K/JailBreakV_28k/"
df_raw = pd.read_csv(os.path.join(base_data_path, "JailBreakV_28K.csv"))
df_raw.head()

## 3. Process each image using Llava model

### 3.1 Using Llava model via Huggingface¶

**!!! Select A100 environment on Colab for best performance**

In [ ]:
# --- Pydantic Schema and Prompt ---
class ImageContent(BaseModel):
    image_description: str = Field(description="A general summary of the image content.")
    visible_text: list[str] = Field(description="A list of all words or sentences that are visually present as text within the image.")

# UPDATED PROMPT: More strictly demands JSON output
content_prompt = """
Analyze the image and generate a single, valid JSON object that strictly adheres to the following Python Pydantic schema:

class ImageContent(BaseModel):
    image_description: str = Field(description="A general summary of the image content, less than 25 words.")
    visible_text: list[str] = Field(description="A list of all words or sentences present as text in the image, less than 10 unique words.")

Ensure the output is ONLY the JSON object itself, without any introductory text or markdown wrappers like ```json.
"""
# --------------------------------------------------------

# --- Global Model and Processor Initialization ---

# Check for GPU availability and use it
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == 'cpu':
    print("WARNING: Running on CPU in Colab will be very slow. Ensure you have a GPU runtime enabled.")

# Load the LLaVA model and processor from Hugging Face Hub
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForVision2Seq.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" # Automatically handles model placement on GPU
)

print("Model and processor loaded successfully.")
# --------------------------------------------------------

def extract_json_from_response(response_text):
    """
    Attempts to extract a valid JSON string from potentially messy model output.
    Handles responses wrapped in markdown blocks (```json ... ```) or raw JSON.
    """
    # Regex to find JSON block wrapped in ```json or ```
    match = re.search(r'```(?:json\n)?(.*?)```', response_text, re.DOTALL)
    if match:
        json_str = match.group(1).strip()
    else:
        # Assume the entire response *might* be raw JSON
        json_str = response_text.strip()

    # Try to load the cleaned string as JSON
    try:
        json_object = json.loads(json_str)
        # Re-dump to normalize the string format for Pydantic validation later
        return json.dumps(json_object)
    except json.JSONDecodeError:
        print(f"Failed to decode raw JSON string during extraction.")
        return None


def process_image_with_llava_hf(image_path):
    """
    Processes an image using the loaded HuggingFace LLaVA model via chat template.
    """
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}. Skipping.")
        return None, None

    try:
        # Open the image using PIL
        image = Image.open(image_path).convert("RGB")

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": content_prompt}
                ]
            },
        ]

        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(model.device)

        with torch.no_grad():
            output_tokens = model.generate(
                **inputs,
                max_new_tokens=300, # Increased tokens for safer JSON output
                temperature=0.0,     # Deterministic output
                pad_token_id=processor.tokenizer.eos_token_id
            )

        # Decode the generated tokens (skipping the input prompt part)
        generated_tokens = output_tokens[0][inputs["input_ids"].shape[-1]:]
        raw_response = processor.decode(generated_tokens, skip_special_tokens=True).strip()

        # --- Use the robust JSON parser ---
        json_string = extract_json_from_response(raw_response)

        if json_string:
            try:
                details = ImageContent.model_validate_json(json_string)
                return details.image_description, details.visible_text
            except ValidationError as e:
                print(f"Pydantic validation failed for {image_path}. Error: {e}")
                print(f"Problematic JSON: {json_string[:200]}...")
        else:
            print(f"Could not extract valid JSON from response for {image_path}.")
            print(f"Raw response: {raw_response[:200]}...")

        # Fallback if any parsing/validation fails
        return "Parsing Error/Check Logs", []

    except Exception as e:
        print(f"An error occurred processing {image_path}: {e}")
        time.sleep(1)
        return None, None

# --- Main script with Caching Logic ---
if __name__ == '__main__':

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # !!! Update this path to where your data is located in Google Drive if necessary !!!
    base_data_path = "/content/drive/MyDrive/Colab Notebooks/JailBreakV_28K/JailBreakV_28k/"

    if not os.path.exists(base_data_path):
        print(f"Error: Base data path not found: {base_data_path}")
        exit()

    df_raw = pd.read_csv(os.path.join(base_data_path, "JailBreakV_28K.csv"))

    temp_cache_file = '/content/drive/MyDrive/Colab Notebooks/JailBreakV_28K/JailBreakV_28k/temp_image_llm_results.csv'
    processed_results = []

    print("------------------ Starting image processing with caching...")

    if os.path.exists(temp_cache_file):
        cached_df = pd.read_csv(temp_cache_file)
        cached_df = df_raw[~df_raw['image_path'].isin(cached_paths)]['image_path']
        cached_paths_list = cached_df['img_path'].astype(str).tolist()
        processed_results = cached_df.to_dict(orient='records')

        print(f"Loaded {len(processed_results)} cached results.")
        cached_paths = set(cached_paths_list)
        images_to_process = df_raw[~df_raw['image_path'].isin(cached_paths)]['image_path'].tolist()
    else:
        images_to_process = df_raw['image_path'].tolist()

    for i, image_path in enumerate(images_to_process):
        print(f"Processing {i+1}/{len(images_to_process)}: {image_path}...")

        full_image_path = os.path.join(base_data_path, image_path)

        description, text_list = process_image_with_llava_hf(full_image_path)

        if description is not None:
            result_dict = {
                'img_path': image_path,
                'image_description': description,
                'visible_text': json.dumps(text_list)
            }
            processed_results.append(result_dict)
            pd.DataFrame(processed_results).to_csv(temp_cache_file, index=False)
            print(f"Successfully processed and cached: {image_path}")

    df_results = pd.DataFrame(processed_results)

    df_final = pd.merge(df_raw, df_results, left_on='image_path', right_on='img_path', how='left')
    df_final['visible_text'] = df_final['visible_text'].apply(lambda x: json.loads(x) if pd.notna(x) else [])

    if 'img_path' in df_final.columns and 'image_path' in df_final.columns:
        df_final = df_final.drop(columns=['img_path'])

    print("\n--- Final Merged DataFrame ---")
    print(df_final.head())
    print(f"Total processed images: {len(df_final)}")


### 3.2 (Optional) using Llava via model served on Ollama

### install Colab extension to run terminal in Colab (to run Ollama) 

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

### install Ollama

In [ ]:
!curl https://ollama.ai/install.sh | sh

### Open 2 terminals to run Ollama

In [2]:
%xterm
# then use command as below:
# ollama serve &

UsageError: Line magic function `%xterm` not found.


In [ ]:
%xterm
# then use command as below:
# ollama pull llava

In [ ]:
# install ollama python library
!pip install ollama

In [ ]:
# Clear CUDA Cache to free GPU RAM
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import os
import io
import time
import json
import ollama
import pandas as pd
from PIL import Image
from pydantic import BaseModel, Field

# --- Pydantic Schema and Prompt (from original code) ---
class ImageContent(BaseModel):
    image_description: str = Field(description="A general summary of the image content.")
    visible_text: list[str] = Field(description="A list of all words or sentences that are visually present as text within the image.")

content_prompt = """
Using the image, provide a detailed description. Confine the response to one sentence with less than 25 words, highlighting the main subject's appearance and environment. Example: Input Image: [image of a brown dog] Output: A small, shaggy brown dog with a red collar is sitting on a green lawn with a blue ball nearby.
"""
# --------------------------------------------------------

def resize_image_if_needed(image_path, max_width=600, max_height=400):
    """
    Resizes an image proportionally if it's larger than max_width or max_height,
    and returns the image data as bytes ready for the Ollama API.
    Forces JPEG compression for speed and payload size reduction.
    """
    with Image.open(image_path) as img:
        width, height = img.size

        if width > max_width or height > max_height:
            # Calculate new dimensions while maintaining aspect ratio
            img.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)
            print(f"Resized image from {width}x{height} to {img.size[0]}x{img.size[1]}")

        # Convert image to bytes in memory as JPEG for efficiency
        img_byte_arr = io.BytesIO()
        # Save as JPEG with moderate quality (85 is good balance of size/quality)
        img.save(img_byte_arr, format='JPEG', quality=85)
        return img_byte_arr.getvalue()

def process_image_with_llava(image_path):
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}. Skipping.")
        return None, None

    try:
        # Get the resized image bytes
        image_bytes = resize_image_if_needed(image_path)

        response = ollama.chat(
            model='llava',
            messages=[{
                'role': 'user',
                'content': content_prompt,
                'images': [image_bytes]
            }],
            format=ImageContent.model_json_schema(),
            options={
                "temperature": 0.0,
                # Explicitly set CPU options for best performance on CPU-only setup
                "num_threads": os.cpu_count(), # Use all available CPU cores
                # "num_ctx": 2048 # Adjust if you have memory issues, default is often fine
            }
        )

        json_output_str = response['message']['content']
        details = ImageContent.model_validate_json(json_output_str)

        return details.image_description, details.visible_text

    except ollama.ResponseError as e:
        print(f"Ollama API error for {image_path}: {e}")
        # If Ollama server is overloaded, waiting and retrying might help
        time.sleep(10)
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred processing {image_path}: {e}")
        return None, None

# --- Main script with Caching Logic (Remains mostly the same but added imports) ---
if __name__ == '__main__':
    # Make sure Ollama server is running (e.g., 'ollama run llava' in your terminal)

    from google.colab import drive
    drive.mount('/content/drive')

    # !!! Update this path to where your data is located in Google Drive !!!
    base_data_path = "/content/drive/MyDrive/Colab Notebooks/JailBreakV_28K/JailBreakV_28k/"

    if not os.path.exists(base_data_path):
        print(f"Error: Base data path not found: {base_data_path}")
        exit()

    df_raw = pd.read_csv(os.path.join(base_data_path, "JailBreakV_28K.csv"))

    temp_cache_file = 'temp_image_llm_results.csv'
    processed_results = []

    print("------------------ Starting image processing with caching...")
    print(f"Detected {os.cpu_count()} CPU cores available for Ollama.")

    # Load existing cache if it exists to resume processing
    if os.path.exists(temp_cache_file):
        cached_df = pd.read_csv(temp_cache_file)
        # Ensure column type consistency before set operations
        cached_paths_list = cached_df['img_path'].astype(str).tolist()
        processed_results = cached_df.to_dict(orient='records')
        print(f"Loaded {len(processed_results)} cached results.")

        # Determine which images still need processing
        cached_paths = set(cached_paths_list)
        images_to_process = df_raw[~df_raw['image_path'].isin(cached_paths)]['image_path'].tolist()
    else:
        images_to_process = df_raw['image_path'].tolist()

    # Process remaining images
    for i, image_path in enumerate(images_to_process):
        print(f"Processing {i+1}/{len(images_to_process)}: {image_path}...")
        # Assuming the relative path is correct based on your previous code
        full_image_path = base_data_path + image_path
        description, text_list = process_image_with_llava(full_image_path)

        if description is not None:
            result_dict = {
                'img_path': image_path,
                'image_description': description,
                'visible_text': json.dumps(text_list)
            }
            processed_results.append(result_dict)
            # Save intermittently
            pd.DataFrame(processed_results).to_csv(temp_cache_file, index=False)
            print(f"Successfully processed and cached: {image_path}")

    df_results = pd.DataFrame(processed_results)
    df_final = pd.merge(df_raw, df_results, on='img_path', how='left')
    df_final['visible_text'] = df_final['visible_text'].apply(lambda x: json.loads(x) if pd.notna(x) else [])
    print("\n--- Final Merged DataFrame ---")
    print(df_final.head())
